# Using GoogleMaps Api to search for facility name and Sta

In [50]:
import pandas as pd
from geopy.geocoders import GoogleV3
import python_creds

# Load the Excel file
df = pd.read_excel('facilities_state.xlsx')

# Initialize the geolocator
geolocator = GoogleV3(api_key=python_creds.google_api)

# Define a function to geocode a single row
def geocode_row(row):
    location = f"{row['Facility']}, {row['State']}"
    address = geolocator.geocode(location)
    return address.address if address else "Address not found"

# Batch size for saving results
batch_size = 1000

# Loop over the DataFrame in batches of size `batch_size`
for i, start_idx in enumerate(range(0, len(df), batch_size)):
    batch = df.iloc[start_idx:start_idx+batch_size]
    df.loc[start_idx:start_idx+batch_size, 'Address'] = batch.apply(geocode_row, axis=1)
    filename = f"facilities_GoogleMapsAPI_{i*batch_size+1}-{(i+1)*batch_size}.xlsx"
    df.iloc[start_idx:start_idx+batch_size].to_excel(filename, index=False)
    print(f"Processed {(i+1)*batch_size} rows...")

Processed 0 rows...
Processed 1000 rows...
Processed 2000 rows...
Processed 3000 rows...
Processed 4000 rows...
Processed 5000 rows...


# Using MapQuest Api

In [ ]:
import requests
import pandas as pd
import math

# Define the MapQuest API key
key = python_creds.MapQuest_api

# Read the facilities Excel file
facilities_df = pd.read_excel('facilities_state.xlsx')

# Define the batch size and calculate the number of batches
batch_size = 1000
num_batches = math.ceil(len(facilities_df) / batch_size)

# Iterate over the batches of facilities and search for their addresses
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    batch_df = facilities_df.iloc[start_idx:end_idx]
    
    # Define the list of addresses for the current batch
    addresses = []
    
    # Iterate over the facilities in the current batch and search for their addresses
    for index, row in batch_df.iterrows():
        query = f'{row["Facility"]} {row["State"]}'
        url = f'http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={query}'
        response = requests.get(url)
        if response.ok:
            json_data = response.json()
            if json_data['results']:
                location = json_data['results'][0]['locations'][0]
                address = location['street'] + ', ' + location['adminArea5'] + ', ' + location['adminArea3'] + ' ' + location['postalCode']
                addresses.append(address)
            else:
                addresses.append('Could not find the address.')
        else:
            addresses.append('Request failed.')
    
    # Save the list of addresses to a new Excel file for the current batch
    batch_num = i + 1
    filename = f'facilities_Mapquest_State{batch_num * batch_size}.xlsx'
    batch_df = batch_df.assign(Address=addresses)
    batch_df.to_excel(filename, index=False)


## Using GoogleMaps Api to Only Search for the Facility Name instead of Facility & State Combination

In [54]:
import pandas as pd
from geopy.geocoders import GoogleV3
import python_creds

# Load the Excel file
df = pd.read_excel('Fac_Not_Found.xlsx')

# Initialize the geolocator
geolocator = GoogleV3(api_key=python_creds.google_api)

# Define a function to geocode a single row
def geocode_row(row):
    location = f"{row['Facility']}"
    address = geolocator.geocode(location)
    return address.address if address else "Address not found"

# Batch size for saving results
batch_size = 1708

# Loop over the DataFrame in batches of size `batch_size`
for i, start_idx in enumerate(range(0, len(df), batch_size)):
    batch = df.iloc[start_idx:start_idx+batch_size]
    df.loc[start_idx:start_idx+batch_size, 'Address'] = batch.apply(geocode_row, axis=1)
    filename = f"facilities_GoogleMapsAPI_NF_{i*batch_size+1}-{(i+1)*batch_size}.xlsx"
    df.iloc[start_idx:start_idx+batch_size].to_excel(filename, index=False)
    print(f"Processed {(i+1)*batch_size} rows...")

Processed 1708 rows...


# Combine the Seperate Batch Documents into 1

In [53]:
import pandas as pd
import glob

# Get a list of all the Excel files with facility addresses
file_list = glob.glob("facilities_GoogleMapsApi_*.xlsx")

# Combine the Excel files into a single DataFrame
df_list = []
for file in file_list:
    df_list.append(pd.read_excel(file))
df = pd.concat(df_list)

# Save the combined DataFrame to a new Excel file
df.to_excel("facilities_GoogleMaps_combined(Original).xlsx", index=False)
